In [293]:
import json
import re
try:  
    from asciitree import LeftAligned
    from collections import OrderedDict as OD
except ModuleNotFoundError:
    print("Error -- Use the requirement.txt to install the dependencies")

### Listing the top 10 files in the directory tables

In [187]:
dirpath = os.path.join( os.getcwd() , "tables")
files = os.listdir(path=dirpath)[:]

schema_table= []
for file in files:
    schema_table.append ((file.split(".")[0],file.split(".")[1]))
schema_table.sort(key=lambda x: x[0])
    





crosscheck.games
crosscheck.calibration_maintenance
crosscheck.tags
games.metadata
games.vision
games.oem
games.vision_oem
games.autorecalibration
games.latency
games.live_final
games.nulls
locations.latency
locations.vision_oem
report.scheduled_no_exceptions_hist
report.hardball_pitchers
report.delivered_new_pk
report.delivered_exceptions_hist
report.verified_levels_18
report.delivered_no_exceptions
report.delivered_combined_hist
report.practice_pitching
report.false_delivered_hist
report.mlb_orgs
report.schedule_new_level
report.scheduled_no_exceptions
report.exp_access_rules_hist
report.delivered_no_exceptions_hist
report.scheduled_combined
report.practice_pitching_dev
report.exp_access_rules
report.scheduled_exceptions
rundown.access_rules_history
rundown.location_history
rundown.access_rules_rows
rundown.location_history_rows
scout.tags
trout.app_angle


In [199]:
def getTheFromClauseText(filepath):
    try:
        fileObj = open(filepath)
        config_dict = json.load(fileObj)
        from_text = None
        try:
            # Fetch the from clause
            from_text = config_dict["query"]["L"][0]["M"]["from"]["S"] # For query with list
        except:
            try:
                from_text = config_dict["query"]["M"]["from"]["S"] # Single query without list
            except:
                # Do nothing if this fails too, and return from_text as None
                pass
        fileObj.close()
        return from_text
    except FileNotFoundError:
        return None


## Loop to get all the text 

In [166]:
# for file in os.listdir(path=dirpath):
#     filepath = os.path.join(dirpath,file)
#     from_text = getTheFromClauseText(filepath)
#     print(from_text)
#     print("\n\n\n")
    


In [201]:
def getTheFirstSource(from_text):
    if from_text != None:
        p = re.compile(r"""
            (?P<starting>\w+[.]\w+) # Get the starting text    
            """, re.VERBOSE)
        m = p.search(from_text)
        return(m.group('starting'))
    else:
        return None
        

from_text = "report.schedule_new_level s left join games.metadata g on s.game_id = g.game_id left join report.exp_access_rules_hist a on (a.level is null or a.level = s.level_name) and (a.location is null or s.location_id = a.location) and (a.team is null or s.home_team_id = a.team)"
print(getTheFirstSource(from_text))

report.schedule_new_level


In [202]:
def getTheJoinTablesList(from_text):
    if from_text !=None:
        p = re.compile(r"""
        join\s+(?P<table>\w+[.]\w+)
        """, re.VERBOSE)
        m = p.findall(from_text)
        return m
    else:
        return None
from_text = "report.schedule_new_level s left join games.metadata g on s.game_id = g.game_id left join report.exp_access_rules_hist a on (a.level is null or a.level = s.level_name) and (a.location is null or s.location_id = a.location) and (a.team is null or s.home_team_id = a.team)"
print(getTheJoinTablesList(from_text))



['games.metadata', 'report.exp_access_rules_hist']


In [220]:
filepath = os.path.join(dirpath,'games.nulls.json')
from_text = getTheFromClauseText(filepath)

print(from_text)



# filepath = os.path.join(dirpath,'base.games.json')
# from_text = getTheFromClauseText(filepath)
# print(from_text)


crosscheck.tags t


In [353]:
class CreateBranch:
    def __init__(self,filepath):
        self.parentNode = None
        self.from_text= None
        self.tables_list = []
        self.tree_dict = {}  
        self.starting_source = None
        self.joining_source= None   
        
        self.getParentNode(filepath)
        self.getTheFromClauseText(filepath)
        self.getTheFirstSource()
        self.getTheJoinTablesList()
        self.consolidateTableList()
        self.construct_tree()
        
    def getParentNode(self,filepath):
        file_exists = os.path.isfile(filepath)
        
        if file_exists:
            self.parentNode = ".".join(os.path.basename(filepath).split(".")[:-1])
        else:
            print("File path doesnot exists")
            
        
    
    def construct_tree(self):
        t_list=[]
        for i in self.tables_list:
            t_list.append((i,{}))
        self.tree_dict[self.parentNode] = OD(t_list)
        tr = LeftAligned()
        print(tr(self.tree_dict))
    
    
    def getTheFromClauseText(self,filepath):
        try:
            fileObj = open(filepath)
            config_dict = json.load(fileObj)
            try:
                # Fetch the from clause
                self.from_text = config_dict["query"]["L"][0]["M"]["from"]["S"] # For query with list
            except:
                try:
                    self.from_text = config_dict["query"]["M"]["from"]["S"] # Single query without list
                except:
                    # Do nothing if this fails too, and return from_text as None
                    pass
            fileObj.close()
        except FileNotFoundError:
            return None
        
    def getTheFirstSource(self):
        
        if self.from_text != None:

            p = re.compile(r"""
                (?P<starting>\w+[.]\w+) # Get the starting text    
                """, re.VERBOSE)
            
            m = p.search(self.from_text)
            self.starting_source = m.group('starting')
            
    def getTheJoinTablesList(self):
        if self.from_text !=None:
            p = re.compile(r"""
            join\s+(?P<table>\w+[.]\w+)
            """, re.VERBOSE)
            self.joining_source = p.findall(self.from_text)
            
    def consolidateTableList(self):
        self.tables_list.append(self.starting_source)
        self.tables_list= self.tables_list + self.joining_source
        
            
filepath = os.path.join(dirpath,'report.hardball_pitchers.json')

c = CreateBranch(filepath)


report.hardball_pitchers
 +-- scout.tags
 +-- base.measurements
 +-- base.games


In [354]:
c.tree_dict

{'report.hardball_pitchers': OrderedDict([('scout.tags', {}),
              ('base.measurements', {}),
              ('base.games', {})])}

In [355]:
c.tables_list

['scout.tags', 'base.measurements', 'base.games']

In [356]:
c.parentNode

'report.hardball_pitchers'

In [357]:
c.tree_dict

{'report.hardball_pitchers': OrderedDict([('scout.tags', {}),
              ('base.measurements', {}),
              ('base.games', {})])}

In [280]:
runLoop = True 
while runLoop:
    
    c = CreateBranch(filepath)
    
    for filename in c.tree_dict

[]

AttributeError: 'CreateBranch' object has no attribute 'current_schema_table'

In [257]:
v.tables_list

[]